#### Fine Tune the gpt2 on colllege corpus text data for next word predictions.

In [1]:
from datasets import load_dataset
# Load your college corpus dataset (text format)
dataset = load_dataset('text', data_files={'train': 'row_data/corpus_data/college_corpus_train.txt','validation': 'row_data/corpus_data/college_corpus_train.txt'})


/home/rahulbharti/Preojects/college-chatbot-gpt2/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# len(dataset["train"]["text"][4])
dataset["train"]["text"][0:5]


['Rajkiya Engineering College (R.E.C.) Ambedkar Nagar was established by Government of Uttar Pradesh under special component plan in year 2010, the college has started offering B.Tech Programme in three disciplines – Information Technology (IT), Electrical Engineering (EE) and Civil Engineering (CE) with intake of 60 seats in each branches from the session 2010-11.',
 'Rajkiya Engineering College, Ambedkar Nagar is one of the best and most reputable government engineering colleges in the state of Uttar Pradesh. REC Ambedkar Nagar has always been excelling both on the academic and the non-academic fronts. Rajkiya Engineering College, Ambedkar Nagar is an AICTE-approved government engineering college with a well-established library and labs. Every year a large number of students from the college clear the GATE exam and get placed into PSUs. ',
 'The work of college construction started in October 2010 with a budget of 6213.81Lakh. The college was running in the campus of Kamla Nehru Inst

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load tokenizer and model, and move the model to the appropriate device
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

tokenizer.pad_token = tokenizer.eos_token
# Resize token embeddings (if adding new tokens specific to college jargon, etc.)
# model.resize_token_embeddings(len(tokenizer))


In [21]:

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=768)

# Tokenize the datasets
# tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [22]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 52
})

In [23]:
for line in tokenized_datasets["train"]["text"][0:5]:
    print(line)

Rajkiya Engineering College (R.E.C.) Ambedkar Nagar was established by Government of Uttar Pradesh under special component plan in year 2010, the college has started offering B.Tech Programme in three disciplines – Information Technology (IT), Electrical Engineering (EE) and Civil Engineering (CE) with intake of 60 seats in each branches from the session 2010-11.
Rajkiya Engineering College, Ambedkar Nagar is one of the best and most reputable government engineering colleges in the state of Uttar Pradesh. REC Ambedkar Nagar has always been excelling both on the academic and the non-academic fronts. Rajkiya Engineering College, Ambedkar Nagar is an AICTE-approved government engineering college with a well-established library and labs. Every year a large number of students from the college clear the GATE exam and get placed into PSUs. 
The work of college construction started in October 2010 with a budget of 6213.81Lakh. The college was running in the campus of Kamla Nehru Institue of Te

In [24]:
from transformers import DataCollatorForLanguageModeling

# Define data collator for next-word prediction (causal language modeling)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set mlm=False because GPT-2 does not use masked language modeling
)


In [26]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
)


In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [53]:
import torch


# Prepare input text
input_text = "Information Technlogy seats"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)  # Move input_ids to the same device

# Generate the next word prediction
outputs = model.generate(input_ids, max_length=500, num_return_sequences=1)

# Decode and print the output
print("_" * 80)
print(f"Input: {input_text}")
print("Output:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


________________________________________________________________________________
Input: Information Technlogy seats
Output:
Information Technlogy seats 60 students, with a capacity of 60. Information Technology is managed by an office located at 5100 N.E. State Drabhumiya Road, Ambedkar Nagar. The office is open from 9.00 A.M. to 5.00P.M. Monday to Friday. Information Technology is also available through the office's Student Information Technology Cell. Information Technology is also available through the office's Student Information Technology Centre. The campus is well equipped with internet, telephone, and library facilities. The campus is also well equipped with computers, printers, and other electronic equipment. The campus is well equipped with a wide variety of computers, including the latest versions of Windows, Linux, and Mac OS X. The campus is also well equipped with a wide variety of sound systems, including a prerecorded system for students. The campus is well equipped wit

In [23]:
tokenizer.encode("rahul", return_tensors='pt')

tensor([[11392,   377]])

In [21]:
tokenizer.decode([278])

'ing'